In [2]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal


pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, get_activations
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line
from sparse_coding.spacy_tag import make_spacy_feature_df


%reload_ext autoreload
%autoreload 2

In [3]:
import subprocess
subprocess.run(['python', '-m', 'spacy', 'download', 'en_core_web_trf'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 10.2 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


CompletedProcess(args=['python', '-m', 'spacy', 'download', 'en_core_web_trf'], returncode=0)

In [4]:
haystack_utils.clean_cache()
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.2"

model_name = "tiny-stories-2L-33M"
print_name = "TinyStories 2L 33M"

model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)

prompts = load_tinystories_validation_prompts(data_path='data/tinystories')[:40]
tokens = model.to_tokens(prompts)
df = make_spacy_feature_df(model, tokens)

(INFO) 12:12:42: Loaded 21990 TinyStories validation prompts


Loaded pretrained model tiny-stories-2L-33M into HookedTransformer
Starting spacy processing of dataset...
Finished spacy processing of dataset.
1701303187.363483 0


In [5]:
df.head()

,is_spacy_adj,is_spacy_adp,is_spacy_det,is_spacy_noun,is_spacy_pron,is_spacy_propn,is_spacy_punct,is_spacy_verb,is_spacy_space,is_spacy_num,...,is_NORP,is_FAC,is_ORDINAL,is_MONEY,is_LOC,is_EVENT,is_LANGUAGE,is_stop,sent_end,sent_begin
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# look for encoder features that go on particular spacy attributes
save_name = '18_morning_sun'
encoder, cfg = load_encoder(save_name, model_name, model, save_path='/workspace')
acts = []
for prompt in prompts:
    acts.append(get_acts(encoder, cfg, save_name, prompts, model))
acts = torch.stack(acts)
print(acts.shape)
# acts, indices = torch.reshape(acts, [acts.shape[0] + acts.shape[1], acts.shape[2]])

from sklearn.metrics import f1_score

threshold = 0.3
f1_scores = {}

spacy_columns = ["attr1"]
for series_name, series in df.items():
    # Binarize neuron activations
    neuron_binarized = df[acts] > threshold
    score = f1_score(series, neuron_binarized)
    f1_scores[series_name] = score
    print(score)


AttributeError: 'tuple' object has no attribute 'shape'